<a href="https://colab.research.google.com/github/Sagnik220/Iris-Classification-GRADIO/blob/main/Iris_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import gradio as gr
from pycaret.datasets import get_data
from pycaret.classification import *

In [5]:
# load the iris dataset from pycaret repo
data = get_data('iris')

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [6]:
# initialize setup
s = setup(data, target = 'species', session_id = 123)

,Description,Value
0,session_id,123
1,Target,species
2,Target Type,Multiclass
3,Label Encoded,"Iris-setosa: 0, Iris-versicolor: 1, Iris-virgi..."
4,Original Data,"(150, 5)"
5,Missing Values,False
6,Numeric Features,4
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [7]:
best = compare_models(n_select = 15)
compare_model_results = pull()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.9809,0.9969,0.9833,0.9857,0.9809,0.9715,0.9739,0.022
knn,K Neighbors Classifier,0.9800,0.9830,0.9806,0.9800,0.9800,0.9697,0.9697,0.118
qda,Quadratic Discriminant Analysis,0.9709,0.9969,0.9750,0.9782,0.9709,0.9566,0.9602,0.023
lr,Logistic Regression,0.9609,0.9921,0.9611,0.9622,0.9596,0.9403,0.9422,0.360
nb,Naive Bayes,0.9609,0.9938,0.9611,0.9652,0.9605,0.9407,0.9432,0.022
gbc,Gradient Boosting Classifier,0.9609,0.9782,0.9611,0.9723,0.9579,0.9402,0.9476,0.257
dt,Decision Tree Classifier,0.9509,0.9616,0.9500,0.9598,0.9479,0.9249,0.9309,0.021
et,Extra Trees Classifier,0.9509,0.9890,0.9528,0.9532,0.9509,0.9258,0.9269,0.471
rf,Random Forest Classifier,0.9409,0.9875,0.9417,0.9422,0.9396,0.9100,0.9119,0.481
ada,Ada Boost Classifier,0.9409,0.9895,0.9417,0.9467,0.9391,0.9100,0.9146,0.118


In [8]:
# creating a predict function to be passed into gradio UI
def predict(model, sepal_length, sepal_width, petal_length, petal_width):
  
    df = pd.DataFrame.from_dict({'sepal_length': [sepal_length], 'sepal_width': [sepal_width], 
                                 'petal_length': [petal_length], 'petal_width': [petal_width]})
    
    model_index = list(compare_model_results['Model']).index(model)
    model = best[model_index]
    pred = predict_model(model, df, raw_score=True)
    return {'Iris-setosa': pred['Score_Iris-setosa'][0].astype('float64'), 
            'Iris-versicolor': pred['Score_Iris-versicolor'][0].astype('float64'),
            'Iris-virginica' : pred['Score_Iris-virginica'][0].astype('float64')}

In [9]:
model = gr.inputs.Dropdown(list(compare_model_results['Model']), label="Model")
sepal_length = gr.inputs.Slider(minimum=1, maximum=10, default=data['sepal_length'].mean(), label = 'sepal_length')
sepal_width = gr.inputs.Slider(minimum=1, maximum=10, default=data['sepal_width'].mean(), label = 'sepal_width')
petal_length = gr.inputs.Slider(minimum=1, maximum=10, default=data['petal_length'].mean(), label = 'petal_length')
petal_width = gr.inputs.Slider(minimum=1, maximum=10, default=data['petal_width'].mean(), label = 'petal_width')

gr.Interface(predict, [model,sepal_length,sepal_width,petal_length,petal_width], "label", live=True).launch()


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://11545.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://11545.gradio.app')